In [4]:
from torf import Torrent
import hashlib
import struct
from math import ceil
import bencodepy
import os

In [2]:
import hashlib


def _read_piece(torrent, piece_idx):
    if torrent.mode == "singlefile":
        offset = piece_idx * torrent.piece_size
        try:
            with open(torrent.name, "rb") as file:
                file.seek(offset)
                data = file.read(torrent.piece_size)
                if not data:
                    return None
                return data
        except FileNotFoundError as e:
            print(f"File not found: {e}")
            return None
        except IOError as e:
            print(f"Error reading file: {e}")
            return None

    else:
        # Handle multi-file mode (if needed)
        pass


torrent = Torrent.read("compnet.torrent")

print(torrent.name)

data = _read_piece(torrent, 2)
print(len(data))
hash = hashlib.sha1(data).digest()
if hash == torrent.hashes[2]:
    print("Piece hash matches")

ReadError: compnet.torrent: No such file or directory

In [ ]:
import os

torrent = Torrent.read("./torrents/asm1.torrent")

info_hash = torrent.infohash


def _check_local_repo(info_hash):

    torrent_files = [f for f in os.listdir("./torrents/") if f.endswith(".torrent")]

    print(torrent_files)

    for torrent_file in torrent_files:

        torrent = Torrent.read("./torrents/" + torrent_file)

        if torrent.infohash == info_hash:

            return torrent_file

    return None


_check_local_repo(info_hash=info_hash)

['asm1.torrent', 'compnet.torrent']


'asm1.torrent'

In [ ]:
client_prefix = "-ST0001-"
import random

# random number 12 digits
unique_component = str(random.randint(0, 999999999999)).zfill(12)
id = f"{client_prefix}{unique_component}"

In [ ]:
info_hash = torrent.infohash
type(info_hash)

str

In [ ]:
import struct

print(len(id))
info_hash = torrent.infohash
pstrlen = struct.pack("B", 19)
pstr = b"BitTorrent protocol"
reserved = b"\x00" * 8
info_hash = bytes.fromhex(info_hash)
print(len(info_hash))
handshake = pstrlen + pstr + reserved + info_hash + id.encode("utf-8")

print(handshake)
print(len(handshake))

TypeError: object of type 'builtin_function_or_method' has no len()

In [ ]:
pstrlen = struct.unpack("B", handshake[0:1])[0]
pstr = handshake[1:20]
reserved = handshake[20:28]
info_hash = handshake[28:48]
peer_id = handshake[48:]

print(pstrlen)
print(pstr.decode())
print(reserved.hex())
print(info_hash.hex())
print(peer_id.decode("utf-8"))

print(torrent.infohash)

NameError: name 'struct' is not defined

In [2]:
from PieceManager import PieceManager
from PeerCommunicator import PeerCommunicator
from torf import Torrent

torrent = Torrent.read("./torrents/compnet.torrent")

pieceManager = PieceManager(torrent)
peerCommunicator = PeerCommunicator()

bitfield = pieceManager.generate_bitfield("./download_path/compnet.pdf")
for bit in bitfield:
    if not bit:
        print("failed")

In [1]:
from utils import get_id

print(get_id())

-ST0001-699544607774


In [13]:
file_path = "./download_path/asm1.pdf"
print(os.listdir("./download_path/"))
with open(file_path, "r+b") as f:
    data = f.read()
print(data)

['asm1.pdf', 'compnet.pdf']
b''
